# Software Defects Analysis & Prediction

## Step 0: Imports

In [1]:
import polars as pl
import seaborn as sb
import dotenv
import os
from pathlib import Path

dotenv.load_dotenv()

True

## Step 1: Load Data

In [2]:
filedir = os.getenv("DATA_DIR") or ""
filename = os.getenv("DATASET_NAME") or ""
filedir = f'.\\..\\{filedir}'

df = pl.read_csv(Path(filedir).joinpath(filename))

df

LOC,CYCLO,LENGTH,VOLUME,DIFFICULTY,INT_FAN_IN,INT_FAN_OUT,NUM_OPERATORS,NUM_OPERANDS,BRANCH_COUNT,DEFECT_LABEL
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0.779239,0.478261,0.274048,0.544918,0.564121,0.222222,0.444444,0.736196,0.807377,0.642857,0
0.595156,0.608696,0.742561,0.758597,0.450649,0.222222,0.0,0.576687,0.20082,0.142857,0
0.895502,0.0,0.968166,0.754277,0.672996,1.0,0.0,0.116564,0.020492,0.0,1
0.782007,0.565217,0.164706,0.017766,0.584106,0.0,1.0,0.615542,0.481557,0.5,0
0.757785,0.217391,0.5609,0.126125,0.52605,0.555556,0.222222,0.656442,0.655738,0.857143,1
…,…,…,…,…,…,…,…,…,…,…
0.577855,0.73913,0.331488,0.09195,0.603102,1.0,0.444444,0.460123,0.081967,0.214286,0
0.500346,0.304348,0.719031,0.553453,0.51187,0.666667,1.0,0.010225,0.713115,0.0,0
0.155017,0.869565,0.843253,0.641748,0.059249,1.0,0.666667,0.435583,0.848361,0.928571,1


## Step 2: Data exploration

In [4]:
print(f'Shape: {df.shape}  ({df.shape[0]} rows x {df.shape[1]} columns)\n')

print('Schema:')
for name, dtype in df.schema.items():
    print(f'  {name:20s} {str(dtype)}')

Shape: (1000, 11)  (1000 rows x 11 columns)

Schema:
  LOC                  Float64
  CYCLO                Float64
  LENGTH               Float64
  VOLUME               Float64
  DIFFICULTY           Float64
  INT_FAN_IN           Float64
  INT_FAN_OUT          Float64
  NUM_OPERATORS        Float64
  NUM_OPERANDS         Float64
  BRANCH_COUNT         Float64
  DEFECT_LABEL         Int64


In [5]:
print('Descriptive statistics:\n')
df.describe()

Descriptive statistics:



statistic,LOC,CYCLO,LENGTH,VOLUME,DIFFICULTY,INT_FAN_IN,INT_FAN_OUT,NUM_OPERATORS,NUM_OPERANDS,BRANCH_COUNT,DEFECT_LABEL
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.520711,0.493304,0.509783,0.505971,0.502349,0.503556,0.510667,0.493988,0.512625,0.514643,0.326
"""std""",0.289402,0.301158,0.289705,0.291389,0.284572,0.315975,0.3214,0.295131,0.276572,0.314337,0.468982
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.278893,0.217391,0.255363,0.252745,0.263143,0.222222,0.222222,0.231084,0.280738,0.214286,0.0
"""50%""",0.53218,0.478261,0.513841,0.522599,0.511867,0.555556,0.555556,0.494888,0.518443,0.571429,0.0
"""75%""",0.767474,0.73913,0.761592,0.759421,0.750672,0.777778,0.777778,0.756646,0.741803,0.785714,1.0
"""max""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
null_counts = df.null_count()
print('Null counts per column:')
print(null_counts)

n_duplicates = df.shape[0] - df.unique().shape[0]
print(f'\nDuplicate rows: {n_duplicates}')


Null counts per column:
shape: (1, 11)
┌─────┬───────┬────────┬────────┬───┬───────────────┬──────────────┬──────────────┬──────────────┐
│ LOC ┆ CYCLO ┆ LENGTH ┆ VOLUME ┆ … ┆ NUM_OPERATORS ┆ NUM_OPERANDS ┆ BRANCH_COUNT ┆ DEFECT_LABEL │
│ --- ┆ ---   ┆ ---    ┆ ---    ┆   ┆ ---           ┆ ---          ┆ ---          ┆ ---          │
│ u32 ┆ u32   ┆ u32    ┆ u32    ┆   ┆ u32           ┆ u32          ┆ u32          ┆ u32          │
╞═════╪═══════╪════════╪════════╪═══╪═══════════════╪══════════════╪══════════════╪══════════════╡
│ 0   ┆ 0     ┆ 0      ┆ 0      ┆ … ┆ 0             ┆ 0            ┆ 0            ┆ 0            │
└─────┴───────┴────────┴────────┴───┴───────────────┴──────────────┴──────────────┴──────────────┘

Duplicate rows: 0


In [7]:
class_counts = df.group_by('DEFECT_LABEL').len().sort('DEFECT_LABEL')
print('Class distribution:')
print(class_counts)

total = df.shape[0]
for row in class_counts.iter_rows():
    label, count = row
    print(f'  Class {label}: {count} ({count/total*100:.1f}%)')

# Flag imbalance
majority = class_counts['len'].max()
minority = class_counts['len'].min()
ratio = minority / majority
print(f'\nImbalance ratio (minority/majority): {ratio:.3f}')
if ratio < 0.43:  # roughly 30/70
    print('Dataset is moderately imbalanced — stratified splitting recommended.')
else:
    print('Class distribution is reasonably balanced.')


Class distribution:
shape: (2, 2)
┌──────────────┬─────┐
│ DEFECT_LABEL ┆ len │
│ ---          ┆ --- │
│ i64          ┆ u32 │
╞══════════════╪═════╡
│ 0            ┆ 674 │
│ 1            ┆ 326 │
└──────────────┴─────┘
  Class 0: 674 (67.4%)
  Class 1: 326 (32.6%)

Imbalance ratio (minority/majority): 0.484
Class distribution is reasonably balanced.


In [9]:
feature_cols = [c for c in df.columns if c != 'DEFECT_LABEL']

print('Mean feature values per class:\n')
per_class_mean = (
    df.group_by('DEFECT_LABEL')
    .agg([pl.col(c).mean().alias(c) for c in feature_cols])
    .sort('DEFECT_LABEL')
)
per_class_mean


Mean feature values per class:



DEFECT_LABEL,LOC,CYCLO,LENGTH,VOLUME,DIFFICULTY,INT_FAN_IN,INT_FAN_OUT,NUM_OPERATORS,NUM_OPERANDS,BRANCH_COUNT
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.522796,0.498581,0.515211,0.513013,0.501868,0.506264,0.512364,0.49064,0.525861,0.500742
1,0.516401,0.482395,0.498562,0.491412,0.503344,0.497955,0.507157,0.50091,0.48526,0.543383


## Step 3: Data visualization

## Step 4: Defects prediction

### Model 1: Logistic Regression

#### The Core Idea

Logistic Regression models the **probability** that an instance belongs to class 1 using the **logistic (sigmoid) function**.

#### Mathematical Formulation

##### 1. Linear Combination

First, compute a linear combination of features:

```
z = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ
z = β₀ + Σ(βᵢ * xᵢ)
```

Where:

- `x₁, x₂, ..., xₙ` are the features
- `β₀, β₁, ..., βₙ` are the coefficients (weights) to be learned
- `β₀` is the intercept (bias)

##### 2. Sigmoid Function

Transform the linear output to a probability using the **sigmoid function**:

```
P(y=1|x) = σ(z) = 1 / (1 + e^(-z))
```

Where:

- `P(y=1|x)` is the probability that y=1 given features x
- `e` is Euler's number (≈2.718)
- The output is always between 0 and 1

**Why sigmoid?** It maps any real number to a range [0, 1]

```
z → -∞    ⟹  P → 0
z = 0     ⟹  P = 0.5
z → +∞    ⟹  P → 1
```

##### 3. Decision Rule

```
Predict class 1 if P(y=1|x) ≥ 0.5
Predict class 0 if P(y=1|x) < 0.5
```

#### Loss Function: Log Loss (Cross-Entropy)

The model learns by minimizing the **log loss**:

```
L(β) = -1/m * Σ[yᵢ * log(P(yᵢ=1|xᵢ)) + (1-yᵢ) * log(1-P(yᵢ=1|xᵢ))]
```

Where:

- `m` is the number of training examples
- `yᵢ` is the actual label (0 or 1)
- `P(yᵢ=1|xᵢ)` is the predicted probability

**Intuition**:

- If actual y=1 and we predict P=0.9, loss is small (-log(0.9) ≈ 0.1)
- If actual y=1 and we predict P=0.1, loss is large (-log(0.1) ≈ 2.3)

#### Regularization

To prevent overfitting, add a penalty term:

**L2 Regularization (Ridge):**

```
L(β) = Log Loss + λ * Σ(βᵢ²)
```

**L1 Regularization (Lasso):**

```
L(β) = Log Loss + λ * Σ|βᵢ|
```

Where `λ` controls regularization strength (larger λ = more regularization).

### Model 2: Random Forest

#### The Core Idea

Build many **decision trees** on random subsets of data and features, then **average** their predictions.

#### Mathematical Formulation

##### 1. Single Decision Tree

A decision tree splits data recursively to maximize **information gain** or minimize **impurity**.

**Gini Impurity** (measure of randomness):

```
Gini(node) = 1 - Σ(pᵢ²)
```

Where:

- `pᵢ` is the proportion of class i samples in the node
- Perfect purity: Gini = 0 (all samples same class)
- Maximum impurity: Gini = 0.5 (equal mix of classes)

**Example:**

```
Node with 60 class-0 and 40 class-1 samples:
p₀ = 60/100 = 0.6
p₁ = 40/100 = 0.4
Gini = 1 - (0.6² + 0.4²) = 1 - 0.52 = 0.48
```

**Information Gain** when splitting:

```
IG = Gini(parent) - Weighted_Average[Gini(left_child), Gini(right_child)]
```

The algorithm chooses splits that **maximize Information Gain**.

##### 2. Bootstrap Aggregating (Bagging)

Random Forest creates diversity through **bagging**:

1. **Randomly sample** m instances from training data (with replacement)
2. **Randomly select** k features at each split (typically k = √n_features)
3. Build a decision tree on this subset
4. Repeat for n_trees

##### 3. Prediction by Voting

For binary classification:

```
P(y=1|x) = 1/N * Σ[tree_i predicts 1]
```

Where N is the number of trees.

**Final prediction:**

```
ŷ = 1 if P(y=1|x) ≥ 0.5, else 0
```

#### Why Does It Work?

**Law of Large Numbers:** Average of many predictions is more stable than individual predictions.

**Bias-Variance Tradeoff:**

- Single tree: Low bias, high variance (overfits)
- Random Forest: Low bias, low variance (averaging reduces variance)

### Model 3: Gradient Boosting

#### The Core Idea

Build trees **sequentially**, where each tree corrects the errors of the previous trees.

#### Mathematical Formulation

##### 1. Additive Model

The prediction is a **sum** of multiple weak learners:

```
F(x) = f₀(x) + η*f₁(x) + η*f₂(x) + ... + η*fₘ(x)
```

Where:

- `F(x)` is the final prediction
- `f₀(x)` is the initial prediction (usually mean)
- `fᵢ(x)` are decision trees (weak learners)
- `η` is the learning rate (0 < η ≤ 1)
- `m` is the number of trees

##### 2. Sequential Learning

At each iteration t:

**Step 1: Calculate residuals (errors)**

```
rᵢ⁽ᵗ⁾ = yᵢ - F⁽ᵗ⁻¹⁾(xᵢ)
```

**Step 2: Fit a new tree to residuals**

```
fₜ(x) ← fit_tree(X, r⁽ᵗ⁾)
```

**Step 3: Update the model**

```
F⁽ᵗ⁾(x) = F⁽ᵗ⁻¹⁾(x) + η * fₜ(x)
```

#### 3. For Binary Classification

Use **log-odds** instead of class labels:

```
F(x) = log(P(y=1|x) / P(y=0|x))
```

Convert to probability:

```
P(y=1|x) = 1 / (1 + e^(-F(x)))
```

##### 4. Gradient Descent Intuition

Gradient Boosting minimizes loss by following the **negative gradient**:

```
fₜ(x) ≈ -∂L/∂F(x)
```

Where L is the loss function (e.g., log loss).

**Why "Gradient" Boosting?** Each new tree fits the **negative gradient** of the loss function.

### Model 4: Support Vector Machine (SVM)

### Model 5: K-Nearest Neighbors (KNN)

### Model 6: Naive Bayes

### Model 7: Neural Network (MLPClassifier)